# Aries Basic Controller - Basic Message Example

In [6]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [7]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

## Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

In [8]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

## Verify an Active Connection Exists

This should have been established through the setup docker container using the create_connection.py script.

In [9]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'request_id': 'f4a2b677-81ab-4d80-a91c-c65983d34d36', 'invitation_mode': 'once', 'created_at': '2020-07-09 12:06:42.378157Z', 'initiator': 'external', 'my_did': 'KNRAwrRgvMgDzkjGarE2cQ', 'connection_id': 'f9586283-be3a-47d0-adf8-4b519e083a5b', 'their_did': 'Ub1Yo5PZ91Rx3cCQr3xfqW', 'updated_at': '2020-07-09 12:06:52.540296Z', 'their_label': 'Bob', 'routing_state': 'none', 'invitation_key': 'GD1LDgWcnME6en4dNbnkp2yvw5ULuUgxuU3HPpuZXmcP', 'accept': 'manual', 'state': 'active'}]
Connection : {'request_id': 'f4a2b677-81ab-4d80-a91c-c65983d34d36', 'invitation_mode': 'once', 'created_at': '2020-07-09 12:06:42.378157Z', 'initiator': 'external', 'my_did': 'KNRAwrRgvMgDzkjGarE2cQ', 'connection_id': 'f9586283-be3a-47d0-adf8-4b519e083a5b', 'their_did': 'Ub1Yo5PZ91Rx3cCQr3xfqW', 'updated_at': '2020-07-09 12:06:52.540296Z', 'their_label': 'Bob', 'routing_state': 'none', 'invitation_key': 'GD1LDgWcnME6en4dNbnkp2yvw5ULuUgxuU3HPpuZXmcP', 'accept': 'manual', 'state': 'active'}
Active Conne

## Send a Basic Message over DIDComm to Bob

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

You can send as messages as you want, if you are running the [basic-message tutorial on Bob's notebook](http://localhost:8889/notebooks/basic-message.ipynb) these will be received and printed by the message handler. You may have to run a code block to see the output.

In [10]:
basic_message = "hello from Alice"
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Alice -> Bob")
print(response)

BASIC MESSAGE - Alice -> Bob
{}


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [11]:
response = await agent_controller.terminate()
print(response)

None
